In [1]:
import pandas as pd
import numpy as np

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test_with_solutions.csv')

In [2]:
#make sure that count(insult) == count(non_insult) in train data
X_train_ins = train_data.loc[train_data['Insult'] == 1]['Comment'].values
y_train_ins = train_data.loc[train_data['Insult'] == 1]['Insult'].values

X_train_non_ins = train_data.loc[train_data['Insult'] == 0]['Comment'].values
y_train_non_ins = train_data.loc[train_data['Insult'] == 0]['Insult'].values

X_train = []
y_train = []
for i in range(len(X_train_ins)):
    X_train.append(X_train_ins[i])
    X_train.append(X_train_non_ins[i])
    
    y_train.append(y_train_ins[i])
    y_train.append(y_train_non_ins[i])


X_test = test_data['Comment'].values
y_test = test_data['Insult'].values

In [3]:
import re

def get_words(line):
    return [x for x in re.split('[_\n ,.?!@#$%^&*():;"\'\/\\0-9\{2}\\\]', line.lower().strip()) if x != '']

def append_words_from_lines(words, lines):
    for line in lines:
        for word in get_words(line):
            if word not in words and word:
                words.append(word)

vocab = []

append_words_from_lines(vocab, train_data['Comment'].values)
append_words_from_lines(vocab, test_data['Comment'].values)

In [4]:
len(vocab)

21289

In [8]:
print(vocab[:300])

['you', 'fuck', 'your', 'dad', 'i', 'really', 'don', 't', 'understand', 'point', 'xa', 'it', 'seems', 'that', 'are', 'mixing', 'apples', 'and', 'oranges', 'a', 'xc', 'majority', 'of', 'canadians', 'can', 'has', 'been', 'wrong', 'before', 'now', 'will', 'be', 'again', 'n', 'nunless', 're', 'supportive', 'the', 'idea', 'nothing', 'is', 'full', 'proof', 'or', 'perfect', 'so', 'take', 'chances', 'if', 'we', 'should', 'inadvertently', 'kill', 'son', 'daughter', 'then', 'them', 's', 'breaks', 'always', 'regard', 'as', 'collateral', 'damage', 'like', 'in', 'wartime', 'sorry', 'but', 'cheques', 'mail', 'listen', 'dont', 'wanna', 'get', 'married', 'to', 'man', 'women', 'do', 'what', 'would', 'bother', 'gay', 'people', 'got', 'stay', 'lane', 'let', 'god', 'nice', 'quick', 'judg', 'not', 'him', 'thought', 'wasnt', 'suppose', 'judge', 'c', 'xe', 'b', 'u', 'ea', 'xu', 'ed', 'ng', 'eddng', 'bi', 'ec', 'xecnh', 'xf', 'ho', 'kh', 'nc', 'd', 'cu', 'xed', 'chi', 'ee', 'nh', 'gi', 'eef', 'edf', 'v', 'gia

In [9]:
import word2vecReader
import numpy as np

w2v = word2vecReader.Word2Vec.load_word2vec_format('word2vec_twitter_model/word2vec_twitter_model.bin', binary=True)

In [10]:
word_to_vec = {}
for word in vocab:
    if w2v.__contains__(word):
        word_to_vec[word] = w2v[word]
    else:
        word_to_vec[word] = np.zeros(400)
    

In [13]:
del w2v

NameError: name 'w2v' is not defined

In [26]:
import torch

seq_len = 20


X_train_vec_seq = []
for i, line in enumerate(X_train[::]):
    X_train_vec_seq.append([])
    words = get_words(line)
    for j in range(seq_len):
        if j < len(words):
            X_train_vec_seq[-1].append(word_to_vec[words[j]])
        else:
            X_train_vec_seq[-1].append(np.zeros(400))
X_train_vec_seq = torch.from_numpy(np.array(X_train_vec_seq)).type(torch.FloatTensor)
            
X_test_vec_seq = []
for i, line in enumerate(X_test[::]):
    X_test_vec_seq.append([])
    words = get_words(line)
    for j in range(seq_len):
        if j < len(words):
            X_test_vec_seq[-1].append(word_to_vec[words[j]])
        else:
            X_test_vec_seq[-1].append(np.zeros(400))
X_test_vec_seq = torch.from_numpy(np.array(X_test_vec_seq)).type(torch.FloatTensor)

y_train_torch = torch.from_numpy(np.array(y_train[::]))   
y_test_torch = torch.from_numpy(np.array(test_data['Insult'].values))

In [27]:
print(X_train_vec_seq[1])


-0.0383  0.0020 -0.0446  ...   0.0045  0.0211 -0.0224
 0.0175  0.0939 -0.0591  ...   0.0802 -0.0016 -0.0358
 0.0408  0.0206 -0.0234  ...   0.0331  0.0218 -0.0536
          ...             ⋱             ...          
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
[torch.FloatTensor of size 20x400]



In [28]:
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [29]:
# Hyper Parameters
sequence_length = seq_len
input_size = 400
hidden_size = 128
num_layers = 2
num_classes = 2
batch_size = len(X_train_vec_seq)
num_epochs = 500
learning_rate = 0.01

In [30]:
# RNN Model (Many-to-One)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial states 
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) 
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        
        # Forward propagate RNN
        out, _ = self.lstm(x, (h0, c0))  
        
        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])  
        return out

rnn = RNN(input_size, hidden_size, num_layers, num_classes)

In [31]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)

In [33]:
# Train the Model
for epoch in range(num_epochs):
        tweet = Variable(X_train_vec_seq)
        label = Variable(y_train_torch.view(-1))
    
        optimizer.zero_grad()
        outputs = rnn(tweet)
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()
        
        if (epoch) % 5 == 0:
            print ('Epoch [%d/%d], Loss: %.4f' 
                   %(epoch+1, num_epochs, loss.data[0]))
            
        if loss.data[0] < 0.05:
            print('done')
            break

Epoch [1/500], Loss: 0.6862
Epoch [6/500], Loss: 0.6920
Epoch [11/500], Loss: 0.6827
Epoch [16/500], Loss: 1.0641
Epoch [21/500], Loss: 0.7815
Epoch [26/500], Loss: 0.7048
Epoch [31/500], Loss: 0.6932
Epoch [36/500], Loss: 0.6971
Epoch [41/500], Loss: 0.6927
Epoch [46/500], Loss: 0.6940
Epoch [51/500], Loss: 0.6924
Epoch [56/500], Loss: 0.6926
Epoch [61/500], Loss: 0.6916
Epoch [66/500], Loss: 0.6879
Epoch [71/500], Loss: 0.6779
Epoch [76/500], Loss: 0.6026
Epoch [81/500], Loss: 0.6007
Epoch [86/500], Loss: 0.5291
Epoch [91/500], Loss: 0.4539
Epoch [96/500], Loss: 0.5063
Epoch [101/500], Loss: 0.4396
Epoch [106/500], Loss: 0.4099
Epoch [111/500], Loss: 0.3882
Epoch [116/500], Loss: 0.3607
Epoch [121/500], Loss: 0.3387
Epoch [126/500], Loss: 0.3168
Epoch [131/500], Loss: 0.2965
Epoch [136/500], Loss: 0.2789
Epoch [141/500], Loss: 0.2852
Epoch [146/500], Loss: 0.2863
Epoch [151/500], Loss: 0.2239
Epoch [156/500], Loss: 0.2205
Epoch [161/500], Loss: 0.1822
Epoch [166/500], Loss: 0.1486
Ep

In [44]:
correct = 0
total = 0
positive = 0
correct_positive = 0
should_be_positive = 0

# run model
tweet = Variable(X_test_vec_seq)
label = y_test_torch.view(-1)   
output = rnn(tweet)
_, predicted = torch.max(output.data, 1)


total += label.size(0)
correct += (predicted == label).sum()
positive += predicted.sum()
should_be_positive += label.sum()

for i in range(len(predicted)):
    if predicted[i] == label[i] and label[i] == 1:
        correct_positive += 1
        
        
precision = correct_positive / positive
recall = correct_positive / should_be_positive
f_score = 2 / ( (1 / precision) + (1 / recall) )


print('Test Accuracy of the model: %d %%' % (100 * correct / total)) 
print('Test F-score of the model: %f' % (f_score)) 

Test Accuracy of the model: 76 %
Test F-score of the model: 0.639306
